<a href="https://colab.research.google.com/github/parlakisik/packetinspector/blob/main/ai/trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# prompt: show files in current directory

!ls -al train_data/


total 82956
drwxr-xr-x 2 root root     4096 Apr 10 03:08 .
drwxr-xr-x 1 root root     4096 Apr  9 22:31 ..
-rw-r--r-- 1 root root 28311552 Apr 10 03:08 UNSW-NB15_1.csv.gz
-rw-r--r-- 1 root root 22020096 Apr 10 03:08 UNSW-NB15_2.csv.gz
-rw-r--r-- 1 root root 18874368 Apr 10 03:08 UNSW-NB15_3.csv.gz
-rw-r--r-- 1 root root 15728640 Apr 10 03:08 UNSW-NB15_4.csv.gz


Unzipping files

In [9]:
!gzip -d train_data/*.gz

Importing libraries

In [11]:
# import libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, re, time, math, tqdm, itertools
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.offline as pyo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
import keras
from keras.layers import Conv2D, Conv1D, MaxPooling2D, MaxPooling1D, Flatten, BatchNormalization, Dense
from keras.models import Sequential
from keras.callbacks import CSVLogger, ModelCheckpoint

Loading test data

In [13]:
dfs = []
for i in range(1,5):
    path = 'train_data/UNSW-NB15_{}.csv'
    dfs.append(pd.read_csv(path.format(i), header = None))
combined_data = pd.concat(dfs).reset_index(drop=True)

<ipython-input-13-e5f0f34acda4>:4: DtypeWarning:

Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.

<ipython-input-13-e5f0f34acda4>:4: DtypeWarning:

Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.



In [14]:
combined_data.head()

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,NaN,0


In [18]:
dataset_columns = pd.read_csv('train_data/NUSW-NB15_features.csv',encoding='ISO-8859-1')
combined_data.columns = dataset_columns['Name']

In [19]:
combined_data.head()

Name,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,NaN,0


Data status

In [20]:
combined_data['Label'].value_counts()

Label
0    2218764
1     321283
Name: count, dtype: int64

Adding clean up

In [21]:
combined_data['attack_cat'] = combined_data['attack_cat'].fillna(value='normal').apply(lambda x: x.strip().lower())

In [22]:
combined_data['attack_cat'] = combined_data['attack_cat'].replace('backdoors','backdoor', regex=True).apply(lambda x: x.strip().lower())

In [23]:
combined_data['ct_flw_http_mthd'] = combined_data['ct_flw_http_mthd'].fillna(value=0)

In [24]:
combined_data['is_ftp_login'] = combined_data['is_ftp_login'].fillna(value=0)

In [25]:
combined_data['is_ftp_login'] = np.where(combined_data['is_ftp_login']>1, 1, combined_data['is_ftp_login'])

In [26]:
combined_data['service'] = combined_data['service'].apply(lambda x:"None" if x=='-' else x)

In [27]:
combined_data['ct_ftp_cmd'] = combined_data['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)

In [28]:
combined_data['attack_cat'].nunique()

10

In [29]:
train, test = train_test_split(combined_data,test_size=0.2,random_state=500)
train, val = train_test_split(train,test_size=0.2,random_state=500)